In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# OLD wrong code
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

# Define a custom PyTorch Dataset for video data
class CustomVideoDataset(Dataset):
    def __init__(self, df, window_size=10, stride=5):
        self.df = df
        self.window_size = window_size
        self.stride = stride
        self.video_windows = self.prepare_windows()

    def __len__(self):
        return len(self.video_windows)

    def __getitem__(self, idx):
        frames = self.video_windows[idx]
        print(frames)

        # TODO: Extract labels or other information
        labels = self.extract_labels(idx)

        # Process frames (convert frames to tensors)
        frames_tensor = torch.tensor(frames)

        return frames_tensor, labels
        #labels= tuple (arousal valence)

    def extract_video_info(self, file_path):
        parts = file_path.split('/')
        video_id = parts[-2]
        return video_id

    def prepare_windows(self):
        video_frames = {}
        for _, row in self.df.iterrows():
            file_path = row['path']
            video_id = self.extract_video_info(file_path)
            if video_id not in video_frames:
                video_frames[video_id] = []
            video_frames[video_id].append(file_path)

        video_windows = {}
        for video_id, frames in video_frames.items():
            windows = []
            for i in range(0, len(frames) - self.window_size + 1, self.stride):
                window = frames[i:i + self.window_size]
                windows.append(window)
            video_windows[video_id] = windows

        return video_windows
        #transforn video_windows to a list
        # a = [item for item in video_windows.items()]
        # tmp = []
        # for item in a:
          # for subitem in item:
            # tmp = tmp + subitem из листа листов сделать 1 лист
        # раньше: 1 картинка, теперь 1 окно!
        # след: массив или эррэй с х,у
        # по индексу дефинируем ф, потом строим х, у
        # ассоциировать фреймы со строчками
        # х получается опять массив массивов
        # все это происходит в getitem
        # можно для удобства запихать в отдельные функции

    def extract_labels(self, idx):
        #labels = None
        return labels

# Load your train, dev, and test datasets
train_df = pd.read_csv('/content/drive/MyDrive/data/AFEW-VA_radiant_fog_160_train.csv')
dev_df = pd.read_csv('/content/drive/MyDrive/data/AFEW-VA_radiant_fog_160_dev.csv')
test_df = pd.read_csv('/content/drive/MyDrive/data/AFEW-VA_radiant_fog_160_test.csv')

# Create instances of CustomVideoDataset for train, dev, and test
train_dataset = CustomVideoDataset(train_df)
dev_dataset = CustomVideoDataset(dev_df)
test_dataset = CustomVideoDataset(test_df)

# Create DataLoaders for train, dev, and test datasets
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



ParserError: ignored

In [ ]:
for x,y in train_loader:
  print(x.shape) # shape (32, 10, 256)
  print(y.shape)
  break

torch.Size([32, 10, 256])
torch.Size([32, 2])


In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class CustomVideoDataset(Dataset):
    def __init__(self, df, window_size=10, stride=5):
        self.df = df
        self.window_size = window_size
        self.stride = stride
        self.video_windows, self.labels_windows = self.prepare_windows()

    def __len__(self):
        return len(self.video_windows)

    def __getitem__(self, idx):
        window_frames = self.video_windows[idx]
        embeddings = [self.df.loc[self.df['path'] == frame, self.df.columns[3:]].values.flatten() for frame in window_frames]
        # embeddings shape list:(10, 256)
        embeddings_flat = np.array(embeddings)
        frames_tensor = torch.tensor(embeddings_flat, dtype=torch.float32)

        labels = self.labels_windows[idx]
        labels_tensor = torch.tensor(labels, dtype=torch.float32)

        return frames_tensor, labels_tensor

    def prepare_windows(self):
        video_frames = {}
        labels = {}
        for _, row in self.df.iterrows():
            video_id = self.extract_video_info(row['path'])
            if video_id not in video_frames:
                video_frames[video_id] = []
                labels[video_id] = []
            video_frames[video_id].append(row['path'])
            labels[video_id].append((row['arousal'], row['valence']))

        video_windows = []
        labels_windows = []
        for video_id in video_frames:
            frames = video_frames[video_id]
            label_vals = labels[video_id]
            for i in range(0, len(frames) - self.window_size + 1, self.stride):
                video_windows.append(frames[i:i + self.window_size])
                window_labels = label_vals[i:i + self.window_size]
                avg_arousal = sum([label[0] for label in window_labels]) / len(window_labels)
                avg_valence = sum([label[1] for label in window_labels]) / len(window_labels)
                labels_windows.append((avg_arousal, avg_valence))

        return video_windows, labels_windows

    def extract_video_info(self, file_path):
        parts = file_path.split('/')
        video_id = parts[-2]
        return video_id

# Load your dataframes
train_df = pd.read_csv('/content/drive/MyDrive/AFEW-VA_radiant_fog_160_train.csv')
dev_df = pd.read_csv('/content/drive/MyDrive/AFEW-VA_radiant_fog_160_dev.csv')
test_df = pd.read_csv('/content/drive/MyDrive/AFEW-VA_radiant_fog_160_test.csv')

# Create instances of CustomVideoDataset for train, dev, and test
train_dataset = CustomVideoDataset(train_df)
dev_dataset = CustomVideoDataset(dev_df)
test_dataset = CustomVideoDataset(test_df)

# Create DataLoaders for train, dev, and test datasets
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# Neuer Abschnitt

In [ ]:

def predict_on_dev(model, dev_loader):
  y_valence_true = []
  y_valence_pred = []
  y_arousal_true = []
  y_arousal_pred = []

  model.eval()
  with torch.no_grad():
      for inputs, labels in dev_loader:
          outputs = model(inputs)
          labels_valence = labels[:, 0]
          labels_arousal = labels[:, 1]
          outputs_valence = outputs[:, 0]
          outputs_arousal = outputs[:, 1]

          y_valence_true.extend(labels_valence.cpu().numpy())
          y_valence_pred.extend(outputs_valence.cpu().numpy())
          y_arousal_true.extend(labels_arousal.cpu().numpy())
          y_arousal_pred.extend(outputs_arousal.cpu().numpy())

  # Calculate metrics
  mae_valence = mean_absolute_error(y_valence_true, y_valence_pred)
  rmse_valence = sqrt(mean_squared_error(y_valence_true, y_valence_pred))
  mae_arousal = mean_absolute_error(y_arousal_true, y_arousal_pred)
  rmse_arousal = sqrt(mean_squared_error(y_arousal_true, y_arousal_pred))

  return (mae_valence, rmse_valence, mae_arousal, rmse_arousal)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
import numpy as np

class FullyConnectedNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FullyConnectedNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.tanh(x)  # Apply tanh activation to the output
        return x

# Define the window size and calculate the input dimension
window_size = 10  # Assuming a window size of 10 frames
embeddings_per_frame = 256
input_dim = window_size * embeddings_per_frame  # 10 frames * 256 embeddings per frame

# Define your model and training hyperparameters
hidden_dim = 64
output_dim = 2  # Arousal and Valence
learning_rate = 0.001
epochs = 100

# Instantiate the model
model = FullyConnectedNN(input_dim, hidden_dim, output_dim)

# Define separate loss functions for arousal and valence
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()

    def forward(self, yhat, y):
        return torch.sqrt(self.mse(yhat, y))

criterion_arousal = RMSELoss()
criterion_valence = RMSELoss()

# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define early stopping parameters
patience = 10  # Number of epochs to wait for improvement
min_val_loss = float('inf')
counter = 0  # Counter for epochs without improvement
best_val_loss = float('inf')
best_epoch = 0
best_mae_arousal = float('inf')
best_mae_valence = float('inf')
best_rmse_arousal = float('inf')
best_rmse_valence = float('inf')
stop_training = False

# Training loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)

        # Split predicted values into arousal and valence
        predicted_arousal = outputs[:, 0]
        predicted_valence = outputs[:, 1]

        # Split ground truth labels into arousal and valence
        labels_arousal = labels[:, 0]
        labels_valence = labels[:, 1]

        # Calculate separate losses for arousal and valence
        loss_arousal = criterion_arousal(predicted_arousal, labels_arousal)
        loss_valence = criterion_valence(predicted_valence, labels_valence)
        loss = loss_arousal + loss_valence

        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Predict on dev data using the trained model
    model.eval()
    dev_mae_valence, dev_rmse_valence, dev_mae_arousal, dev_rmse_arousal = predict_on_dev(model, dev_loader)
    general_rmse_metric = (dev_rmse_valence + dev_rmse_arousal) / 2.

    print(f"Validation RMSE: {general_rmse_metric:.4f}")

    # Check Early stopping criteria
    if general_rmse_metric < min_val_loss:  # Check if the validation loss has improved
      min_val_loss = general_rmse_metric
      counter = 0

    # Save the model weights if RMSE is lower than the best value
      #if dev_rmse_arousal < best_rmse_arousal and dev_rmse_valence < best_rmse_valence:
      best_rmse_arousal = dev_rmse_arousal
      best_mae_arousal = dev_mae_arousal
      best_rmse_valence = dev_rmse_valence
      best_mae_valence = dev_mae_valence
      best_epoch = epoch
      torch.save(model.state_dict(), 'best_model_SEWA.pth') #should be called AFEW actually
    else:
        counter += 1
    # If the validation loss hasn't improved for 'patience' epochs, set the stop_training variable
    if counter >= patience:
        print(f"Early stopping at epoch {epoch + 1} due to no improvement in validation loss.")
        stop_training = True
    if stop_training:
        break

# Print the last best results and epoch
if stop_training:
    print(f"Best RMSE Arousal: {best_rmse_arousal:.4f} at epoch {best_epoch + 1}")
    print(f"Best MAE Arousal: {best_mae_arousal:.4f} at epoch {best_epoch + 1}")
    print(f"Best RMSE Valence: {best_rmse_valence:.4f} at epoch {best_epoch + 1}")
    print(f"Best MAE Valence: {best_mae_valence:.4f} at epoch {best_epoch + 1}")
else:
    print("Training completed without early stopping")

def evaluate_on_test(model, test_loader):
    model.eval()
    y_valence_true = []
    y_valence_pred = []
    y_arousal_true = []
    y_arousal_pred = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            labels_valence = labels[:, 0]
            labels_arousal = labels[:, 1]
            outputs_valence = outputs[:, 0]
            outputs_arousal = outputs[:, 1]

            y_valence_true.extend(labels_valence.cpu().numpy().flatten())
            y_valence_pred.extend(outputs_valence.cpu().numpy().flatten())
            y_arousal_true.extend(labels_arousal.cpu().numpy().flatten())
            y_arousal_pred.extend(outputs_arousal.cpu().numpy().flatten())

    # Calculate metrics
    mae_valence_test = mean_absolute_error(y_valence_true, y_valence_pred)
    rmse_valence_test = sqrt(mean_squared_error(y_valence_true, y_valence_pred))
    mae_arousal_test = mean_absolute_error(y_arousal_true, y_arousal_pred)
    rmse_arousal_test = sqrt(mean_squared_error(y_arousal_true, y_arousal_pred))

    return mae_valence_test, rmse_valence_test, mae_arousal_test, rmse_arousal_test

# Load the best model
model.load_state_dict(torch.load('/content/best_model_SEWA.pth')) #AFEW actually
model.eval()

# Call the evaluate_on_test function
mae_valence_test, rmse_valence_test, mae_arousal_test, rmse_arousal_test = evaluate_on_test(model, test_loader)

# Print the results for the test dataset
print(f"Test MAE Valence: {mae_valence_test:.4f}, Test RMSE Valence: {rmse_valence_test:.4f}")
print(f"Test MAE Arousal: {mae_arousal_test:.4f}, Test RMSE Arousal: {rmse_arousal_test:.4f}")


RuntimeError: mat1 and mat2 shapes cannot be multiplied (320x256 and 2560x64)

In [ ]:
#LSTM:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt

# Custom Dataset
class CustomVideoDataset(Dataset):
    def __init__(self, df, window_size=10, stride=5):
        self.df = df
        self.window_size = window_size
        self.stride = stride
        self.video_windows, self.labels_windows = self.prepare_windows()

    def __len__(self):
        return len(self.video_windows)

    def __getitem__(self, idx):
        window_frames = self.video_windows[idx]
        embeddings = [self.df.loc[self.df['path'] == frame, self.df.columns[3:]].values for frame in window_frames]
        frames_tensor = torch.tensor(embeddings, dtype=torch.float32).squeeze(1)

        labels = self.labels_windows[idx]
        labels_tensor = torch.tensor(labels, dtype=torch.float32)

        return frames_tensor, labels_tensor

    def prepare_windows(self):
        video_frames = {}
        labels = {}
        for _, row in self.df.iterrows():
            video_id = self.extract_video_info(row['path'])
            if video_id not in video_frames:
                video_frames[video_id] = []
                labels[video_id] = []
            video_frames[video_id].append(row['path'])
            labels[video_id].append((row['arousal'], row['valence']))

        video_windows = []
        labels_windows = []
        for video_id in video_frames:
            frames = video_frames[video_id]
            label_vals = labels[video_id]
            for i in range(0, len(frames) - self.window_size + 1, self.stride):
                video_windows.append(frames[i:i + self.window_size])
                window_labels = label_vals[i:i + self.window_size]
                avg_arousal = sum([label[0] for label in window_labels]) / len(window_labels)
                avg_valence = sum([label[1] for label in window_labels]) / len(window_labels)
                labels_windows.append((avg_arousal, avg_valence))

        return video_windows, labels_windows

    def extract_video_info(self, file_path):
        parts = file_path.split('/')
        video_id = parts[-2]
        return video_id

# LSTM Network
class LSTMNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMNetwork, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM Layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True) # GRU instead of LSTM

        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)
        self.tanh = nn.Tanh()

        # for 3. approach conv1d_layer = torch.nn.Conv1d(in_channels=hidden_size, out_channels=hidden_size, kernel_size=seq_length)

    def forward(self, x):
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device) # TODO: delete it
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device) # TODO: delete it

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # We try several models
        # 1. just take last state (out[:, -1, :])
        # 2. Adaptive AVG or MAX pooling (https://pytorch.org/docs/stable/generated/torch.nn.AdaptiveAvgPool2d.html)
        # 3. 1D CNN layer (https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html)

        # for 3. approach
        # torch.permute(out, (0, 2, 1)) (batch_size, seq_length, hidden_size) -> (batch_size, hidden_size, seq_length)
        # out = conv1d_layer(out)
        # out = out.squeeze()

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        out = self.tanh(out)

        return out

# Hyperparameters
window_size = 10
input_size = 256  # Number of features (embeddings) per frame
hidden_size = 128  # Number of features in hidden state of LSTM
output_size = 2  # Output size (arousal and valence)
num_layers = 1  # Number of LSTM layers
learning_rate = 0.001
batch_size = 32
epochs = 10

# Load data
train_df = pd.read_csv('/content/drive/MyDrive/AFEW-VA_radiant_fog_160_train.csv')
dev_df = pd.read_csv('/content/drive/MyDrive/AFEW-VA_radiant_fog_160_dev.csv')
test_df = pd.read_csv('/content/drive/MyDrive/AFEW-VA_radiant_fog_160_test.csv')

# Create datasets and dataloaders
train_dataset = CustomVideoDataset(train_df, window_size)
dev_dataset = CustomVideoDataset(dev_df, window_size)
test_dataset = CustomVideoDataset(test_df, window_size)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model
model = LSTMNetwork(input_size, hidden_size, output_size, num_layers)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation step
    model.eval()
    with torch.no_grad():
        total_val_loss = 0
        for inputs, labels in dev_loader:
            outputs = model(inputs)
            val_loss = criterion(outputs, labels)
            total_val_loss += val_loss.item()
        avg_val_loss = total_val_loss / len(dev_loader)
        print(f"Epoch {epoch+1}/{epochs}, Validation Loss: {avg_val_loss:.4f}")

def evaluate_model(model, test_loader):
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            y_true.append(labels.numpy())
            y_pred.append(outputs.numpy())

    y_true = np.concatenate(y_true, axis=0)
    y_pred = np.concatenate(y_pred, axis=0)

    mae_valence = mean_absolute_error(y_true[:, 0], y_pred[:, 0])
    rmse_valence = sqrt(mean_squared_error(y_true[:, 0], y_pred[:, 0]))
    mae_arousal = mean_absolute_error(y_true[:, 1], y_pred[:, 1])
    rmse_arousal = sqrt(mean_squared_error(y_true[:, 1], y_pred[:, 1]))

    return mae_valence, rmse_valence, mae_arousal, rmse_arousal

# Evaluate the model on test data
mae_valence, rmse_valence, mae_arousal, rmse_arousal = evaluate_model(model, test_loader)

print(f"Test MAE Valence: {mae_valence:.4f}, Test RMSE Valence: {rmse_valence:.4f}")
print(f"Test MAE Arousal: {mae_arousal:.4f}, Test RMSE Arousal: {rmse_arousal:.4f}")



Epoch 1/10, Validation Loss: 9.1029
Epoch 2/10, Validation Loss: 9.1272
Epoch 3/10, Validation Loss: 9.1784
Epoch 4/10, Validation Loss: 9.1269
Epoch 5/10, Validation Loss: 9.1227
Epoch 6/10, Validation Loss: 9.1000
Epoch 7/10, Validation Loss: 9.1187
Epoch 8/10, Validation Loss: 9.1081
Epoch 9/10, Validation Loss: 9.1686
Epoch 10/10, Validation Loss: 9.0963
Test MAE Valence: 2.1868, Test RMSE Valence: 2.9043
Test MAE Arousal: 1.9744, Test RMSE Arousal: 2.7255


In [ ]:
#GRU instead LSTM, last state approach
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt

# Custom Dataset
class CustomVideoDataset(Dataset):
    def __init__(self, df, window_size=10, stride=5):
        self.df = df
        self.window_size = window_size
        self.stride = stride
        self.video_windows, self.labels_windows = self.prepare_windows()

    def __len__(self):
        return len(self.video_windows)

    def __getitem__(self, idx):
        window_frames = self.video_windows[idx]
        embeddings = [self.df.loc[self.df['path'] == frame, self.df.columns[3:]].values for frame in window_frames]
        frames_tensor = torch.tensor(embeddings, dtype=torch.float32).squeeze(1)

        labels = self.labels_windows[idx]
        labels_tensor = torch.tensor(labels, dtype=torch.float32)

        return frames_tensor, labels_tensor

    def prepare_windows(self):
        video_frames = {}
        labels = {}
        for _, row in self.df.iterrows():
            video_id = self.extract_video_info(row['path'])
            if video_id not in video_frames:
                video_frames[video_id] = []
                labels[video_id] = []
            video_frames[video_id].append(row['path'])
            labels[video_id].append((row['arousal'], row['valence']))

        video_windows = []
        labels_windows = []
        for video_id in video_frames:
            frames = video_frames[video_id]
            label_vals = labels[video_id]
            for i in range(0, len(frames) - self.window_size + 1, self.stride):
                video_windows.append(frames[i:i + self.window_size])
                window_labels = label_vals[i:i + self.window_size]
                avg_arousal = sum([label[0] for label in window_labels]) / len(window_labels)
                avg_valence = sum([label[1] for label in window_labels]) / len(window_labels)
                labels_windows.append((avg_arousal, avg_valence))

        return video_windows, labels_windows

    def extract_video_info(self, file_path):
        parts = file_path.split('/')
        video_id = parts[-2]
        return video_id

# GRU Network
class GRUNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(GRUNetwork, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # GRU Layer
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)

        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)
        self.tanh = nn.Tanh()

    def forward(self, x):
        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate GRU
        out, _ = self.gru(x, h0)  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        out = self.tanh(out)

        return out

# Hyperparameters
window_size = 10
input_size = 256  # Number of features (embeddings) per frame
hidden_size = 128  # Number of features in hidden state of GRU
output_size = 2  # Output size (arousal and valence)
num_layers = 1  # Number of GRU layers
learning_rate = 0.001
batch_size = 32
epochs = 10

# Load data
train_df = pd.read_csv('/content/drive/MyDrive/AFEW-VA_radiant_fog_160_train.csv')
dev_df = pd.read_csv('/content/drive/MyDrive/AFEW-VA_radiant_fog_160_dev.csv')
test_df = pd.read_csv('/content/drive/MyDrive/AFEW-VA_radiant_fog_160_test.csv')

# Create datasets and dataloaders
train_dataset = CustomVideoDataset(train_df, window_size)
dev_dataset = CustomVideoDataset(dev_df, window_size)
test_dataset = CustomVideoDataset(test_df, window_size)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model with GRU
model = GRUNetwork(input_size, hidden_size, output_size, num_layers)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation step
    model.eval()
    with torch.no_grad():
        total_val_loss = 0
        for inputs, labels in dev_loader:
            outputs = model(inputs)
            val_loss = criterion(outputs, labels)
            total_val_loss += val_loss.item()
        avg_val_loss = total_val_loss / len(dev_loader)
        print(f"Epoch {epoch+1}/{epochs}, Validation Loss: {avg_val_loss:.4f}")

# Test evaluation function
def evaluate_model(model, test_loader):
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            y_true.append(labels.numpy())
            y_pred.append(outputs.numpy())

    y_true = np.concatenate(y_true, axis=0)
    y_pred = np.concatenate(y_pred, axis=0)

    mae_valence = mean_absolute_error(y_true[:, 0], y_pred[:, 0])
    rmse_valence = sqrt(mean_squared_error(y_true[:, 0], y_pred[:, 0]))
    mae_arousal = mean_absolute_error(y_true[:, 1], y_pred[:, 1])
    rmse_arousal = sqrt(mean_squared_error(y_true[:, 1], y_pred[:, 1]))

    return mae_valence, rmse_valence, mae_arousal, rmse_arousal

# Evaluate the model on test data
mae_valence, rmse_valence, mae_arousal, rmse_arousal = evaluate_model(model, test_loader)

print(f"Test MAE Valence: {mae_valence:.4f}, Test RMSE Valence: {rmse_valence:.4f}")
print(f"Test MAE Arousal: {mae_arousal:.4f}, Test RMSE Arousal: {rmse_arousal:.4f}")


Epoch 1/10, Validation Loss: 9.0849
Epoch 2/10, Validation Loss: 9.0904
Epoch 3/10, Validation Loss: 9.0964
Epoch 4/10, Validation Loss: 9.1111
Epoch 5/10, Validation Loss: 9.1322
Epoch 6/10, Validation Loss: 9.1282
Epoch 7/10, Validation Loss: 9.1163
Epoch 8/10, Validation Loss: 9.1390
Epoch 9/10, Validation Loss: 9.1170
Epoch 10/10, Validation Loss: 9.1371
Test MAE Valence: 2.2106, Test RMSE Valence: 2.9149
Test MAE Arousal: 2.0104, Test RMSE Arousal: 2.7364


In [ ]:
# GRU, AVG Pooling approach
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt

# Custom Dataset
class CustomVideoDataset(Dataset):
    def __init__(self, df, window_size=10, stride=5):
        self.df = df
        self.window_size = window_size
        self.stride = stride
        self.video_windows, self.labels_windows = self.prepare_windows()

    def __len__(self):
        return len(self.video_windows)

    def __getitem__(self, idx):
        window_frames = self.video_windows[idx]
        embeddings = [self.df.loc[self.df['path'] == frame, self.df.columns[3:]].values for frame in window_frames]
        frames_tensor = torch.tensor(embeddings, dtype=torch.float32).squeeze(1)

        labels = self.labels_windows[idx]
        labels_tensor = torch.tensor(labels, dtype=torch.float32)

        return frames_tensor, labels_tensor

    def prepare_windows(self):
        video_frames = {}
        labels = {}
        for _, row in self.df.iterrows():
            video_id = self.extract_video_info(row['path'])
            if video_id not in video_frames:
                video_frames[video_id] = []
                labels[video_id] = []
            video_frames[video_id].append(row['path'])
            labels[video_id].append((row['arousal'], row['valence']))

        video_windows = []
        labels_windows = []
        for video_id in video_frames:
            frames = video_frames[video_id]
            label_vals = labels[video_id]
            for i in range(0, len(frames) - self.window_size + 1, self.stride):
                video_windows.append(frames[i:i + self.window_size])
                window_labels = label_vals[i:i + self.window_size]
                avg_arousal = sum([label[0] for label in window_labels]) / len(window_labels)
                avg_valence = sum([label[1] for label in window_labels]) / len(window_labels)
                labels_windows.append((avg_arousal, avg_valence))

        return video_windows, labels_windows

    def extract_video_info(self, file_path):
        parts = file_path.split('/')
        video_id = parts[-2]
        return video_id
class GRUNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, seq_length=10):
        super(GRUNetwork, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # GRU Layer
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)

        #self.adaptive_avg_pool = nn.AdaptiveAvgPool2d((1, hidden_size))# Adaptive Avg Pooling
        # 1D Average Pooling
        self.avg_pool1d = nn.AvgPool1d(seq_length)

        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)
        self.tanh = nn.Tanh()

    def forward(self, x):
        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate GRU
        out, _ = self.gru(x, h0)  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Apply 1D average pooling
        out = out.permute(0, 2, 1)  # Change shape to (batch_size, hidden_size, seq_length) for AvgPool1d
        out = self.avg_pool1d(out)
        out = out.squeeze(2)  # Remove the last dimension

        # Decode the hidden state
        out = self.fc(out)
        out = self.tanh(out)

        return out

# Hyperparameters
window_size = 10
input_size = 256  # Number of features (embeddings) per frame
hidden_size = 128  # Number of features in hidden state of GRU
output_size = 2  # Output size (arousal and valence)
num_layers = 1  # Number of GRU layers
learning_rate = 0.001
batch_size = 32
epochs = 10

# Load data
train_df = pd.read_csv('/content/drive/MyDrive/AFEW-VA_radiant_fog_160_train.csv')
dev_df = pd.read_csv('/content/drive/MyDrive/AFEW-VA_radiant_fog_160_dev.csv')
test_df = pd.read_csv('/content/drive/MyDrive/AFEW-VA_radiant_fog_160_test.csv')

# Create datasets and dataloaders
train_dataset = CustomVideoDataset(train_df, window_size)
dev_dataset = CustomVideoDataset(dev_df, window_size)
test_dataset = CustomVideoDataset(test_df, window_size)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model with GRU
model = GRUNetwork(input_size, hidden_size, output_size, num_layers)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation step
    model.eval()
    with torch.no_grad():
        total_val_loss = 0
        for inputs, labels in dev_loader:
            outputs = model(inputs)
            val_loss = criterion(outputs, labels)
            total_val_loss += val_loss.item()
        avg_val_loss = total_val_loss / len(dev_loader)
        print(f"Epoch {epoch+1}/{epochs}, Validation Loss: {avg_val_loss:.4f}")

# Test evaluation function
def evaluate_model(model, test_loader):
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            y_true.append(labels.numpy())
            y_pred.append(outputs.numpy())

    y_true = np.concatenate(y_true, axis=0)
    y_pred = np.concatenate(y_pred, axis=0)

    mae_valence = mean_absolute_error(y_true[:, 0], y_pred[:, 0])
    rmse_valence = sqrt(mean_squared_error(y_true[:, 0], y_pred[:, 0]))
    mae_arousal = mean_absolute_error(y_true[:, 1], y_pred[:, 1])
    rmse_arousal = sqrt(mean_squared_error(y_true[:, 1], y_pred[:, 1]))

    return mae_valence, rmse_valence, mae_arousal, rmse_arousal

# Evaluate the model on test data
mae_valence, rmse_valence, mae_arousal, rmse_arousal = evaluate_model(model, test_loader)

print(f"Test MAE Valence: {mae_valence:.4f}, Test RMSE Valence: {rmse_valence:.4f}")
print(f"Test MAE Arousal: {mae_arousal:.4f}, Test RMSE Arousal: {rmse_arousal:.4f}")



<ipython-input-3-f5082fb22f33>:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  frames_tensor = torch.tensor(embeddings, dtype=torch.float32).squeeze(1)


Epoch 1/10, Validation Loss: 9.1426
Epoch 2/10, Validation Loss: 9.1051
Epoch 3/10, Validation Loss: 9.1180
Epoch 4/10, Validation Loss: 9.1184
Epoch 5/10, Validation Loss: 9.1250
Epoch 6/10, Validation Loss: 9.1272
Epoch 7/10, Validation Loss: 9.2115
Epoch 8/10, Validation Loss: 9.1271
Epoch 9/10, Validation Loss: 9.1267
Epoch 10/10, Validation Loss: 9.1150
Test MAE Valence: 2.2398, Test RMSE Valence: 2.9325
Test MAE Arousal: 1.9690, Test RMSE Arousal: 2.7247


In [ ]:
from torchsummary import summary
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
class GRUNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, seq_length=10):
        super(GRUNetwork, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # GRU Layer
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)

        #self.adaptive_avg_pool = nn.AdaptiveAvgPool2d((1, hidden_size))# Adaptive Avg Pooling
        # 1D Average Pooling
        self.avg_pool1d = nn.AvgPool1d(seq_length)

        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)
        self.tanh = nn.Tanh()

    def forward(self, x):
        print(x.shape)
        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate GRU
        out, _ = self.gru(x, h0)  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Apply 1D average pooling
        print(out.shape)
        out = out.permute(0, 2, 1)  # Change shape to (batch_size, hidden_size, seq_length) for AvgPool1d
        out = self.avg_pool1d(out)
        out = out.squeeze(2)  # Remove the last dimension
        print(out.shape)

        # Decode the hidden state
        out = self.fc(out)
        out = self.tanh(out)

        return out
# Hyperparameters
window_size = 10
input_size = 256  # Number of features (embeddings) per frame
hidden_size = 128  # Number of features in hidden state of GRU
output_size = 2  # Output size (arousal and valence)
num_layers = 1  # Number of GRU layers
learning_rate = 0.001
batch_size = 32
epochs = 10
# Initialize the model with GRU
model = GRUNetwork(input_size, hidden_size, output_size, num_layers)
summary(model, (10,256))

torch.Size([2, 10, 256])
torch.Size([2, 10, 128])
torch.Size([2, 128])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
               GRU-1  [[-1, 10, 128], [-1, 2, 128]]               0
         AvgPool1d-2               [-1, 128, 1]               0
            Linear-3                    [-1, 2]             258
              Tanh-4                    [-1, 2]               0
Total params: 258
Trainable params: 258
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 2.50
Params size (MB): 0.00
Estimated Total Size (MB): 2.51
----------------------------------------------------------------


In [ ]:
#GRU, MAX Pooling approach
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt

# Custom Dataset
class CustomVideoDataset(Dataset):
    def __init__(self, df, window_size=10, stride=5):
        self.df = df
        self.window_size = window_size
        self.stride = stride
        self.video_windows, self.labels_windows = self.prepare_windows()

    def __len__(self):
        return len(self.video_windows)

    def __getitem__(self, idx):
        window_frames = self.video_windows[idx]
        embeddings = [self.df.loc[self.df['path'] == frame, self.df.columns[3:]].values for frame in window_frames]
        frames_tensor = torch.tensor(embeddings, dtype=torch.float32).squeeze(1)

        labels = self.labels_windows[idx]
        labels_tensor = torch.tensor(labels, dtype=torch.float32)

        return frames_tensor, labels_tensor

    def prepare_windows(self):
        video_frames = {}
        labels = {}
        for _, row in self.df.iterrows():
            video_id = self.extract_video_info(row['path'])
            if video_id not in video_frames:
                video_frames[video_id] = []
                labels[video_id] = []
            video_frames[video_id].append(row['path'])
            labels[video_id].append((row['arousal'], row['valence']))

        video_windows = []
        labels_windows = []
        for video_id in video_frames:
            frames = video_frames[video_id]
            label_vals = labels[video_id]
            for i in range(0, len(frames) - self.window_size + 1, self.stride):
                video_windows.append(frames[i:i + self.window_size])
                window_labels = label_vals[i:i + self.window_size]
                avg_arousal = sum([label[0] for label in window_labels]) / len(window_labels)
                avg_valence = sum([label[1] for label in window_labels]) / len(window_labels)
                labels_windows.append((avg_arousal, avg_valence))

        return video_windows, labels_windows

    def extract_video_info(self, file_path):
        parts = file_path.split('/')
        video_id = parts[-2]
        return video_id

class GRUNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(GRUNetwork, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # GRU Layer
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)

        # Max Pooling
        self.max_pool = nn.MaxPool2d((1, hidden_size))

        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)
        self.tanh = nn.Tanh()

    def forward(self, x):
        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate GRU
        out, _ = self.gru(x, h0)  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Apply max pooling
        out = out.unsqueeze(1)  # Add a channel dimension for pooling
        out = self.max_pool(out)
        out = out.squeeze()  # Remove extra dimensions

        # Decode the hidden state
        out = self.fc(out)
        out = self.tanh(out)

        return out

# Hyperparameters
window_size = 10
input_size = 256  # Number of features (embeddings) per frame
hidden_size = 128  # Number of features in hidden state of GRU
output_size = 2  # Output size (arousal and valence)
num_layers = 1  # Number of GRU layers
learning_rate = 0.001
batch_size = 32
epochs = 10

# Load data
train_df = pd.read_csv('/content/drive/MyDrive/AFEW-VA_radiant_fog_160_train.csv')
dev_df = pd.read_csv('/content/drive/MyDrive/AFEW-VA_radiant_fog_160_dev.csv')
test_df = pd.read_csv('/content/drive/MyDrive/AFEW-VA_radiant_fog_160_test.csv')

# Create datasets and dataloaders
train_dataset = CustomVideoDataset(train_df, window_size)
dev_dataset = CustomVideoDataset(dev_df, window_size)
test_dataset = CustomVideoDataset(test_df, window_size)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model with GRU
model = GRUNetwork(input_size, hidden_size, output_size, num_layers)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation step
    model.eval()
    with torch.no_grad():
        total_val_loss = 0
        for inputs, labels in dev_loader:
            outputs = model(inputs)
            val_loss = criterion(outputs, labels)
            total_val_loss += val_loss.item()
        avg_val_loss = total_val_loss / len(dev_loader)
        print(f"Epoch {epoch+1}/{epochs}, Validation Loss: {avg_val_loss:.4f}")

# Test evaluation function
def evaluate_model(model, test_loader):
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            y_true.append(labels.numpy())
            y_pred.append(outputs.numpy())

    y_true = np.concatenate(y_true, axis=0)
    y_pred = np.concatenate(y_pred, axis=0)

    mae_valence = mean_absolute_error(y_true[:, 0], y_pred[:, 0])
    rmse_valence = sqrt(mean_squared_error(y_true[:, 0], y_pred[:, 0]))
    mae_arousal = mean_absolute_error(y_true[:, 1], y_pred[:, 1])
    rmse_arousal = sqrt(mean_squared_error(y_true[:, 1], y_pred[:, 1]))

    return mae_valence, rmse_valence, mae_arousal, rmse_arousal

# Evaluate the model on test data
mae_valence, rmse_valence, mae_arousal, rmse_arousal = evaluate_model(model, test_loader)

print(f"Test MAE Valence: {mae_valence:.4f}, Test RMSE Valence: {rmse_valence:.4f}")
print(f"Test MAE Arousal: {mae_arousal:.4f}, Test RMSE Arousal: {rmse_arousal:.4f}")


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x10 and 128x2)

In [ ]:
#GRU, 1D CNN Layer
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt

# Custom Dataset
class CustomVideoDataset(Dataset):
    def __init__(self, df, window_size=10, stride=5):
        self.df = df
        self.window_size = window_size
        self.stride = stride
        self.video_windows, self.labels_windows = self.prepare_windows()

    def __len__(self):
        return len(self.video_windows)

    def __getitem__(self, idx):
        window_frames = self.video_windows[idx]
        embeddings = [self.df.loc[self.df['path'] == frame, self.df.columns[3:]].values for frame in window_frames]
        frames_tensor = torch.tensor(embeddings, dtype=torch.float32).squeeze(1)

        labels = self.labels_windows[idx]
        labels_tensor = torch.tensor(labels, dtype=torch.float32)

        return frames_tensor, labels_tensor

    def prepare_windows(self):
        video_frames = {}
        labels = {}
        for _, row in self.df.iterrows():
            video_id = self.extract_video_info(row['path'])
            if video_id not in video_frames:
                video_frames[video_id] = []
                labels[video_id] = []
            video_frames[video_id].append(row['path'])
            labels[video_id].append((row['arousal'], row['valence']))

        video_windows = []
        labels_windows = []
        for video_id in video_frames:
            frames = video_frames[video_id]
            label_vals = labels[video_id]
            for i in range(0, len(frames) - self.window_size + 1, self.stride):
                video_windows.append(frames[i:i + self.window_size])
                window_labels = label_vals[i:i + self.window_size]
                avg_arousal = sum([label[0] for label in window_labels]) / len(window_labels)
                avg_valence = sum([label[1] for label in window_labels]) / len(window_labels)
                labels_windows.append((avg_arousal, avg_valence))

        return video_windows, labels_windows

    def extract_video_info(self, file_path):
        parts = file_path.split('/')
        video_id = parts[-2]
        return video_id

class GRUNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, seq_length=10):
        super(GRUNetwork, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # GRU Layer
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)

        # 1D CNN Layer
        self.conv1d_layer = nn.Conv1d(in_channels=hidden_size, out_channels=hidden_size, kernel_size=seq_length)

        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)
        self.tanh = nn.Tanh()

    def forward(self, x):
        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate GRU
        out, _ = self.gru(x, h0)  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Apply 1D CNN
        out = out.permute(0, 2, 1)  # Change shape to (batch_size, hidden_size, seq_length) for Conv1d
        out = self.conv1d_layer(out)
        out = out.squeeze(2)  # Remove the last dimension

        # Decode the hidden state
        out = self.fc(out)
        out = self.tanh(out)

        return out

# Hyperparameters
window_size = 10
input_size = 256  # Number of features (embeddings) per frame
hidden_size = 128  # Number of features in hidden state of GRU
output_size = 2  # Output size (arousal and valence)
num_layers = 1  # Number of GRU layers
learning_rate = 0.001
batch_size = 32
epochs = 10

# Load data
train_df = pd.read_csv('/content/drive/MyDrive/AFEW-VA_radiant_fog_160_train.csv')
dev_df = pd.read_csv('/content/drive/MyDrive/AFEW-VA_radiant_fog_160_dev.csv')
test_df = pd.read_csv('/content/drive/MyDrive/AFEW-VA_radiant_fog_160_test.csv')

# Create datasets and dataloaders
train_dataset = CustomVideoDataset(train_df, window_size)
dev_dataset = CustomVideoDataset(dev_df, window_size)
test_dataset = CustomVideoDataset(test_df, window_size)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model with GRU
model = GRUNetwork(input_size, hidden_size, output_size, num_layers)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation step
    model.eval()
    with torch.no_grad():
        total_val_loss = 0
        for inputs, labels in dev_loader:
            outputs = model(inputs)
            val_loss = criterion(outputs, labels)
            total_val_loss += val_loss.item()
        avg_val_loss = total_val_loss / len(dev_loader)
        print(f"Epoch {epoch+1}/{epochs}, Validation Loss: {avg_val_loss:.4f}")

# Test evaluation function
def evaluate_model(model, test_loader):
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            y_true.append(labels.numpy())
            y_pred.append(outputs.numpy())

    y_true = np.concatenate(y_true, axis=0)
    y_pred = np.concatenate(y_pred, axis=0)

    mae_valence = mean_absolute_error(y_true[:, 0], y_pred[:, 0])
    rmse_valence = sqrt(mean_squared_error(y_true[:, 0], y_pred[:, 0]))
    mae_arousal = mean_absolute_error(y_true[:, 1], y_pred[:, 1])
    rmse_arousal = sqrt(mean_squared_error(y_true[:, 1], y_pred[:, 1]))

    return mae_valence, rmse_valence, mae_arousal, rmse_arousal

# Evaluate the model on test data
mae_valence, rmse_valence, mae_arousal, rmse_arousal = evaluate_model(model, test_loader)

print(f"Test MAE Valence: {mae_valence:.4f}, Test RMSE Valence: {rmse_valence:.4f}")
print(f"Test MAE Arousal: {mae_arousal:.4f}, Test RMSE Arousal: {rmse_arousal:.4f}")


<ipython-input-6-14f02bfa2486>:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  frames_tensor = torch.tensor(embeddings, dtype=torch.float32).squeeze(1)


Epoch 1/10, Validation Loss: 9.1674
Epoch 2/10, Validation Loss: 9.1590
Epoch 3/10, Validation Loss: 9.1049
Epoch 4/10, Validation Loss: 9.1288
Epoch 5/10, Validation Loss: 9.1284
Epoch 6/10, Validation Loss: 9.1222
Epoch 7/10, Validation Loss: 9.1142
Epoch 8/10, Validation Loss: 9.1572
Epoch 9/10, Validation Loss: 9.1566
Epoch 10/10, Validation Loss: 9.1251
Test MAE Valence: 2.2061, Test RMSE Valence: 2.9216
Test MAE Arousal: 1.9929, Test RMSE Arousal: 2.7300
